In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install schedulefree optuna -q

In [ ]:
train=pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/train.csv")
test=pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/test.csv")
sub=pd.read_csv("/kaggle/input/eds-232-ocean-chemistry-prediction-for-calcofi/sample_submission.csv")

In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import random

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Drop unneeded columns and handle missing values
train = train.drop(columns=["Unnamed: 12", "id"])  # Dropping unnecessary columns

# CRITICAL FIX: Rename TA1.x to TA1 to match test data
train = train.rename(columns={"TA1.x": "TA1"})

# Find common columns between train and test (excluding "DIC" from train)
common_columns = train.drop(columns=["DIC"]).columns.intersection(test.columns)

print(f"Common columns: {len(common_columns)}")
print(f"Common columns list: {sorted(common_columns.tolist())}")

# Select the common columns for both train and test
X = train[common_columns]
y = train["DIC"]
test = test[common_columns]

# Normalize X and y separately
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()
test_scaled = scaler_X.transform(test)

# Split the training data
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=SEED)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class OceanChemistryDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        # Handle both numpy arrays and pandas Series
        if hasattr(y, 'values'):
            y = y.values
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = OceanChemistryDataset(X_train, y_train)
val_dataset = OceanChemistryDataset(X_val, y_val)

# Set generator for reproducible shuffling
g = torch.Generator()
g.manual_seed(SEED)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, generator=g)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

In [ ]:
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_layers=[128, 64], dropout_rate=0.3):
        super(MLPModel, self).__init__()
        
        # Create layers dynamically
        self.layers = nn.ModuleList()
        # self.batch_norms = nn.ModuleList()
        self.activations = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        
        # Input layer
        prev_size = input_size
        
        # Hidden layers
        for hidden_size in hidden_layers:
            # Linear layer
            fc = nn.Linear(prev_size, hidden_size)
            # He initialization for ReLU
            nn.init.kaiming_normal_(fc.weight, mode='fan_in', nonlinearity='relu')
            nn.init.constant_(fc.bias, 0)
            
            self.layers.append(fc)
            # self.batch_norms.append(nn.BatchNorm1d(hidden_size))
            self.activations.append(nn.ReLU())
            self.dropouts.append(nn.Dropout(dropout_rate))
            
            prev_size = hidden_size
        
        # Output layer
        self.output = nn.Linear(prev_size, 1)
        # Xavier initialization for output layer
        nn.init.xavier_normal_(self.output.weight)
        nn.init.constant_(self.output.bias, 0)

    def forward(self, x):
        # Hidden layers with ReLU and Dropout
        for fc, activation, dropout in zip(self.layers, self.activations, self.dropouts):
            x = fc(x)
            # x = bn(x)  # BatchNorm disabled
            x = activation(x)
            x = dropout(x)  # Dropout for regularization
        
        # Output layer (no dropout)
        x = self.output(x)
        return x

# Set device (GPU if available, else CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Initialize the model with custom hidden layer sizes and dropout
hidden_layers = [128, 64]  # You can change this to [256, 128, 64] or any configuration
dropout_rate = 0.3  # Dropout rate (0.3 = 30% of neurons dropped during training)
model = MLPModel(input_size=X_train.shape[1], hidden_layers=hidden_layers, dropout_rate=dropout_rate)
model = model.to(device)  # Move model to GPU

print(f"Input size: {X_train.shape[1]}")
print(f"Hidden layers: {hidden_layers}")
print(f"Dropout rate: {dropout_rate}")
print(f"Model architecture: {X_train.shape[1]} -> {' -> '.join(map(str, hidden_layers))} -> 1")

In [ ]:
import torch.optim as optim
# import schedulefree

# Loss function and optimizer
criterion = nn.MSELoss()

# Use standard AdamW optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=0.001,
    betas=(0.9, 0.999),
    weight_decay=0.01  # L2 regularization
)

# Use Schedule-Free AdamW optimizer (commented out)
# optimizer = schedulefree.AdamWScheduleFree(
#     model.parameters(),
#     lr=0.001,
#     betas=(0.9, 0.999),
#     weight_decay=0
# )

# Training function
def train_model(model, train_loader, val_loader, epochs=5000):
    for epoch in range(epochs):
        model.train()
        # optimizer.train()  # Only for Schedule-Free
        running_loss = 0.0
        train_predictions = []
        train_targets = []

        for X_batch, y_batch in train_loader:
            # Move data to GPU
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs.squeeze(), y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            # Store predictions and targets for RMSE calculation (move to CPU)
            train_predictions.extend(outputs.squeeze().detach().cpu().numpy())
            train_targets.extend(y_batch.cpu().numpy())

        # Inverse transform to original scale
        train_predictions_original = scaler_y.inverse_transform(np.array(train_predictions).reshape(-1, 1)).flatten()
        train_targets_original = scaler_y.inverse_transform(np.array(train_targets).reshape(-1, 1)).flatten()
        
        # Calculate train metrics in original scale
        train_mae = np.mean(np.abs(train_predictions_original - train_targets_original))
        train_rmse = np.sqrt(np.mean((train_predictions_original - train_targets_original)**2))
        
        val_loss = 0.0
        val_predictions = []
        val_targets = []
        model.eval()
        # optimizer.eval()  # Only for Schedule-Free
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                # Move data to GPU
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                
                outputs = model(X_batch)
                loss = criterion(outputs.squeeze(), y_batch)
                val_loss += loss.item()
                
                # Store predictions and targets for RMSE calculation (move to CPU)
                val_predictions.extend(outputs.squeeze().cpu().numpy())
                val_targets.extend(y_batch.cpu().numpy())
        
        # Inverse transform to original scale
        val_predictions_original = scaler_y.inverse_transform(np.array(val_predictions).reshape(-1, 1)).flatten()
        val_targets_original = scaler_y.inverse_transform(np.array(val_targets).reshape(-1, 1)).flatten()
        
        # Calculate validation metrics in original scale
        val_mae = np.mean(np.abs(val_predictions_original - val_targets_original))
        val_rmse = np.sqrt(np.mean((val_predictions_original - val_targets_original)**2))

        if epoch % 100 == 0:
            print(f"Epoch {epoch+1}/{epochs} | "
                  f"Train MAE: {train_mae:.2f}, Train RMSE: {train_rmse:.2f} | "
                  f"Val MAE: {val_mae:.2f}, Val RMSE: {val_rmse:.2f}")

# Train the model
train_model(model, train_loader, val_loader, epochs=5000)

In [ ]:
# ============================================
# Optuna Hyperparameter Optimization
# ============================================
# Uncomment below to run Optuna search

"""
import optuna
from optuna.trial import Trial

def objective(trial: Trial):
    # Hyperparameters to optimize
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-1, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    
    # Hidden layers configuration
    n_layers = trial.suggest_int('n_layers', 1, 4)
    hidden_layers = []
    for i in range(n_layers):
        hidden_size = trial.suggest_categorical(f'hidden_size_{i}', [32, 64, 128, 256, 512])
        hidden_layers.append(hidden_size)
    
    # Create dataloaders with suggested batch size
    g_trial = torch.Generator()
    g_trial.manual_seed(SEED)
    train_loader_trial = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, generator=g_trial)
    val_loader_trial = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Create model with suggested hyperparameters
    model_trial = MLPModel(input_size=X_train.shape[1], hidden_layers=hidden_layers, dropout_rate=dropout_rate)
    model_trial = model_trial.to(device)
    
    # Create optimizer with suggested hyperparameters
    optimizer_trial = optim.AdamW(model_trial.parameters(), lr=lr, weight_decay=weight_decay)
    criterion_trial = nn.L1Loss()
    
    # Early stopping
    best_val_rmse = float('inf')
    patience = 50
    patience_counter = 0
    
    # Training loop (shorter for Optuna)
    for epoch in range(1000):  # Max 1000 epochs
        # Training
        model_trial.train()
        for X_batch, y_batch in train_loader_trial:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer_trial.zero_grad()
            outputs = model_trial(X_batch)
            loss = criterion_trial(outputs.squeeze(), y_batch)
            loss.backward()
            optimizer_trial.step()
        
        # Validation
        model_trial.eval()
        val_predictions = []
        val_targets = []
        with torch.no_grad():
            for X_batch, y_batch in val_loader_trial:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model_trial(X_batch)
                val_predictions.extend(outputs.squeeze().cpu().numpy())
                val_targets.extend(y_batch.cpu().numpy())
        
        # Calculate RMSE in original scale
        val_predictions_original = scaler_y.inverse_transform(np.array(val_predictions).reshape(-1, 1)).flatten()
        val_targets_original = scaler_y.inverse_transform(np.array(val_targets).reshape(-1, 1)).flatten()
        val_rmse = np.sqrt(np.mean((val_predictions_original - val_targets_original)**2))
        
        # Early stopping
        if val_rmse < best_val_rmse:
            best_val_rmse = val_rmse
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break
        
        # Report intermediate value for pruning
        trial.report(val_rmse, epoch)
        
        # Handle pruning
        if trial.should_prune():
            raise optuna.TrialPruned()
    
    return best_val_rmse

# Create study and optimize
study = optuna.create_study(direction='minimize', study_name='mlp_optimization')
study.optimize(objective, n_trials=50, timeout=3600)  # 50 trials or 1 hour

# Print best parameters
print("\n" + "="*50)
print("Best trial:")
print(f"  RMSE: {study.best_trial.value:.4f}")
print("  Params:")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")
print("="*50)

# Train final model with best parameters
best_params = study.best_trial.params
print("\nTraining final model with best parameters...")
"""

In [ ]:
# Convert the test set into a torch tensor and move to GPU
test_tensor = torch.tensor(test_scaled, dtype=torch.float32).to(device)

# Set the model to evaluation mode
model.eval()

# Make predictions
with torch.no_grad():
    predictions_scaled = model(test_tensor).squeeze().cpu().numpy()
    # Inverse transform to get actual DIC values
    predictions = scaler_y.inverse_transform(predictions_scaled.reshape(-1, 1)).flatten()

print(f"Predictions - Min: {predictions.min():.2f}, Max: {predictions.max():.2f}, Mean: {predictions.mean():.2f}")

# Prepare submission
submission = pd.DataFrame({"id": range(1455, 1455 + len(predictions)), "DIC": predictions})
submission.to_csv("submission.csv", index=False)
print("Submission saved!")